In [ ]:
Advanced Probability Functions
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

.. code-block:: python

   def time_dependent_mobility(distances, time_factor=1.0):
       """
       Mobility that changes based on time of day or simulation conditions.
       """
       import numpy as np
       distances = np.array(distances)
       
       # During rush hour (time_factor > 1), prefer longer commutes
       # During leisure time (time_factor < 1), prefer nearby locations
       base_probs = np.exp(-distances / (1000 * time_factor))
       return base_probs / np.sum(base_probs) if np.sum(base_probs) > 0 else base_probs

   def socioeconomic_mobility(distances, wealth_level=0.5):
       """
       Mobility patterns based on agent characteristics.
       """
       import numpy as np
       distances = np.array(distances)
       
       # Wealthier agents can travel farther more easily
       characteristic_distance = 1000 + (wealth_level * 5000)  # 1-6km range
       probs = np.exp(-distances / characteristic_distance)
       return probs / np.sum(probs) if np.sum(probs) > 0 else probs